# Renewable energy feed-in tariffs and emissions

Assembles a dataset that contains:
- some country specific features
- their renewable energy feed-in tariffs
- their CO2 emissions

In [ ]:
import os
import pandas as pd

## Constants

In [ ]:
# OECD data starts on that year
START_YEAR = 2000
# OECD data ends on that year
END_YEAR = 2019
FIT_PREFIX = "FIT_"

## OECD renewable energy feed-in tariffs

NOTE: Run the [OECD tariffs notebook](../oecd/oecd_tariffs.ipynb) to generate the .csv file.  
It contains country feed-in tariffs (FIT) by renewable energy source

In [ ]:
fit_filename = "../oecd/processed/fit.csv"
fit_df = pd.read_csv(fit_filename)
fit_df.head()

In [ ]:
# Prefix the column to easily identify them
fit_index = ["year", "country", "country_code"]
fit_columns = {"Small Hydro": "Small_Hydro", "Solar PV": "Solar_PV"}

for column in fit_df.columns:
    if column not in fit_index:
            fit_columns[column] = FIT_PREFIX + column
fit_df.rename(fit_columns, axis=1, inplace=True)
fit_df.head()

In [ ]:
# How many countries do we have data for, for each year?
fit_df.groupby("year")["country"].nunique()

In [ ]:
# Only use countries for which we have data for all the years
countries = list(fit_df[fit_df["year"] == END_YEAR].country.unique())
country_codes = list(fit_df[fit_df["year"] == END_YEAR].country_code.unique())
print(f"{len(countries)} countries with data for {END_YEAR}:")
countries

In [ ]:
# Keep only country with data from 2000 to 2019
fit_df = fit_df[fit_df["country_code"].isin(country_codes)]

In [ ]:
fit_df.shape

## World Bank CCDR data

NOTE: Run the [World Bank CCDR notebook](../worldbank/world_bank_ccdr.ipynb) to generate the .csv file.  
It contains country features and emissions from the World Bank Country Climate and Development Report (CCDR)

In [ ]:
ccdr_filename = "../worldbank/processed/ccdr.csv"
ccdr_df = pd.read_csv(ccdr_filename)
ccdr_df.head()

In [ ]:
# Remove data for which we do not have tariffs
ccdr_df = ccdr_df[ccdr_df["year"] >= START_YEAR]
ccdr_df = ccdr_df[ccdr_df["year"] <= END_YEAR]
ccdr_df.head()

### Context features

List of potentially interesting features  

See full list here: https://databank.worldbank.org/source/country-climate-and-development-report-(ccdr)

In [ ]:
# Map of feature name to description
features = {
    "EG.ELC.ACCS.ZS": "Access to electricity (% of population)",
    "AG.LND.AGRI.ZS": "Agricultural land (% of land area)",
    "AG.LND.ARBL.ZS": "Arable land (% of land area)",
    "GC.DOD.TOTL.GD.ZS": "Central government debt, total (% of GDP)",
    "CC.EG.SOLR.KW": "Average practical solar potential (kWh/kWp/day)",
    "CC.EG.WIND.PC": "Offshore wind potential - Per capita (kW/cap)",
    "CC.ELEC.CON": "Electricity net consumption",
    "CC.ELEC.GEN": "Electricity net generation",
    "CC.EG.INTS.KW": "Energy intensity of the economy (kWh per 2011$PPP)",
    "IC.ELC.OUTG.ZS": "Firms experiencing electrical outages (% of firms)",
    "CC.EG.SUBF.PC": "Fossil-fuel pre-tax subsidies (consumption and production) USD per capita",
    "NY.GDP.MKTP.KD.ZG": "GDP growth (annual %)",
    "NY.GDP.PCAP.CD": "GDP per capita (current US$)",
    "CC.GHG.GRPE": "GHG growth (annual %)",
    "SI.POV.GINI": "Gini index (World Bank estimate)",
    "GE.EST": "Government Effectiveness: Estimate",
    "SE.ADT.LITR.ZS": "Literacy rate, adult total (% of people ages 15 and above)",
    "CC.SE.NYRS.AVG": "Mean number of years of education completed, aged 17 and older",
    "CC.SH.AIRP.AMB": "Mortality rate attributable to ambient air pollution (deaths per 100 000 population)",
    "CC.TCFD.COMP.EN": "Number of companies that are TCFD compliant by sector - Energy",
    "CC.FSU.PECA": "Per capita food supply (kcal/cap/day)",
    "CC.GHG.PECA": "Per capita GHG emissions (tons/capita)",
    "CC.SE.CAT1.ZS": "Percentage of population with No Education",
    "CC.SE.CAT2.ZS": "Percentage of population with Primary Education",
    "CC.SE.CAT3.ZS": "Percentage of population with Secondary Education",
    "CC.SE.CAT4.ZS": "Percentage of population with Post Secondary Education",
    "SP.POP.TOTL": "Population, total",
    "RQ.EST": "Regulatory Quality: Estimate",
    "RL.EST": "Rule of Law: Estimate(",
    "AG.LND.FRST.ZS": "Share of surface occupied by forest (% of total)",
    "GC.TAX.TOTL.GD.ZS": "Tax revenue (% of GDP)",
    "SL.UEM.TOTL.ZS": "Unemployment, total (% of total labor force) (modeled ILO estimate)",
    "SP.URB.TOTL.IN.ZS": "Urban population (% of total population)",
    "VA.EST": "Voice and Accountability: Estimate",
}

### Outcomes

In [ ]:
outcomes = {
    "CC.CO2.EMSE.EN": "CO2 emissions by sector (Mt CO2 eq) - Energy",
    "CC.NCO.GHG.EN": "Non-CO2 GHG emissions by sector (Mt CO2 eq) - Energy",
}

### Filter

In [ ]:
index_columns = ["country_code", "country_name", "year"]
keep_columns = index_columns + (list(features.keys()) + list(outcomes.keys()))
ccdr_df = ccdr_df[keep_columns]

## Merge

In [ ]:
# Remove column 'country' from fit_df: we'll get country_name from ccdr_df
fit_df.drop("country", axis=1, inplace=True)

In [ ]:
fit_df.shape

In [ ]:
ccdr_df.shape

In [ ]:
# Remove countries from ccdr_df for which we don't have OECD data
ccdr_df = ccdr_df[ccdr_df["country_code"].isin(country_codes)]

In [ ]:
ccdr_df.shape

In [ ]:
# We now have the same number of rows for each DataFrame: nb countries x nb years
df = pd.merge(ccdr_df, fit_df, on=["country_code", "year"], how='outer')

In [ ]:
df.shape

In [ ]:
df.head()

## Export
Export to .csv file

In [ ]:
processed_dir = "processed"
csv_full_filename = os.path.join(processed_dir, "tariffs_and_emissions.csv")
csv_full_filename

In [ ]:
# Create the output directory if needed
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
df.to_csv(csv_full_filename, encoding='utf-8', index=False)